<h1>Creating the Dataframe</h1>

In [1]:
#importing libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
html=urlopen(url).read()

In [4]:
#finding the tag
html

b'\n<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XptJpgpAICsAAFpdq2oAAAAK","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":951325562,"wgRevisionId":951325562,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related 

In [5]:
#creating a BeautifulSoup object
soup=BeautifulSoup(html, "html.parser")

In [6]:
tags = soup('td')

In [7]:
#finding the last tag's index with only neighborhood data which is required
count = 0
for tag in tags:
    print("Content: ",tag.contents[0])
    print(count)
    count = count +1
#ie 539

Content:  M1A

0
Content:  Not assigned

1
Content:  

2
Content:  M2A

3
Content:  Not assigned

4
Content:  

5
Content:  M3A

6
Content:  North York

7
Content:  Parkwoods

8
Content:  M4A

9
Content:  North York

10
Content:  Victoria Village

11
Content:  M5A

12
Content:  Downtown Toronto

13
Content:  Regent Park / Harbourfront

14
Content:  M6A

15
Content:  North York

16
Content:  Lawrence Manor / Lawrence Heights

17
Content:  M7A

18
Content:  Downtown Toronto

19
Content:  Queen's Park / Ontario Provincial Government

20
Content:  M8A

21
Content:  Not assigned

22
Content:  

23
Content:  M9A

24
Content:  Etobicoke

25
Content:  Islington Avenue

26
Content:  M1B

27
Content:  Scarborough

28
Content:  Malvern / Rouge

29
Content:  M2B

30
Content:  Not assigned

31
Content:  

32
Content:  M3B

33
Content:  North York

34
Content:  Don Mills

35
Content:  M4B

36
Content:  East York

37
Content:  Parkview Hill / Woodbine Gardens

38
Content:  M5B

39
Content:  Downtown 

Content:  <a href="/wiki/Nova_Scotia" title="Nova Scotia">NS</a>
544
Content:  <a href="/wiki/Prince_Edward_Island" title="Prince Edward Island">PE</a>
545
Content:  <a href="/wiki/New_Brunswick" title="New Brunswick">NB</a>
546
Content:  <a href="/wiki/Quebec" title="Quebec">QC</a>
547
Content:  <a href="/wiki/Ontario" title="Ontario">ON</a>
548
Content:  <a href="/wiki/Manitoba" title="Manitoba">MB</a>
549
Content:  <a href="/wiki/Saskatchewan" title="Saskatchewan">SK</a>
550
Content:  <a href="/wiki/Alberta" title="Alberta">AB</a>
551
Content:  <a href="/wiki/British_Columbia" title="British Columbia">BC</a>
552
Content:  <a href="/wiki/Nunavut" title="Nunavut">NU</a>
553
Content:  <a href="/wiki/Yukon" title="Yukon">YT</a>
554
Content:  <a href="/wiki/List_of_postal_codes_of_Canada:_A" title="List of postal codes of Canada: A">A</a>
555
Content:  <a href="/wiki/List_of_postal_codes_of_Canada:_B" title="List of postal codes of Canada: B">B</a>
556
Content:  <a href="/wiki/List_of_po

In [8]:
#defining the dataframe
df = pd.DataFrame({'PostalCode':[], 'Borough':[], 'Neighborhood':[]})
df

,PostalCode,Borough,Neighborhood


In [9]:
i = 0
while True:
    if i>539:
        break
    if tags[i+1].contents[0].strip()=='Not assigned':
        i = i+3
        continue
    if tags[i+2].contents[0]=='\n':
        df = df.append({'PostalCode': tags[i].contents[0].strip(), 'Borough': tags[i+1].contents[0].strip(), 'Neighborhood': tags[i+1].contents[0].strip()}, ignore_index=True)
    else:
        df = df.append({'PostalCode': tags[i].contents[0].strip(), 'Borough': tags[i+1].contents[0].strip(), 'Neighborhood': tags[i+2].contents[0].strip().replace('/',', ')}, ignore_index=True)
    i = i + 3
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill N..."
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea..."


In [10]:
df.shape

(103, 3)

<h1>Getting the lat long of each Neighborhood</h1>

In [11]:
coordinates = pd.read_csv('Data/Geospatial_Coordinates.csv')

In [12]:
coor = pd.DataFrame({"Latitude": [], "Longitude": []})

In [13]:
for i in range(df.shape[0]):
    if df.iloc[i, 0] in coordinates[['Postal Code']].values:
        index = coordinates.index[coordinates['Postal Code']==df.iloc[i, 0]][0]
        coor = coor.append({'Latitude':coordinates.iloc[index, 1], 'Longitude':coordinates.iloc[index, 2]}, ignore_index=True)

In [14]:
coor

,Latitude,Longitude
0,43.753259,-79.329656
1,43.725882,-79.315572
2,43.654260,-79.360636
3,43.718518,-79.464763
4,43.662301,-79.389494
...,...,...
98,43.653654,-79.506944
99,43.665860,-79.383160
100,43.662744,-79.321558
101,43.636258,-79.498509


In [15]:
df_new = pd.concat([df, coor], axis=1)

In [16]:
df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


<h1>Getting venue data for only Boroughs with Toronto in their name</h1>

In [17]:
df_new['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [25]:
df_borough = df_new.loc[(df_new['Borough']=='Downtown Toronto')|(df_new['Borough']=='East Toronto')|(df_new['Borough']=='West Toronto')|(df_new['Borough']=='Central Toronto')]

In [26]:
df_borough.reset_index(inplace = True)

In [27]:
df_borough.shape

(39, 6)

In [28]:
df_borough.drop(['index'], axis=1, inplace = True)
df_borough.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [29]:
CLIENT_ID = 'I removed my credentials' # your Foursquare ID
CLIENT_SECRET = 'I removed my credentials' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Exploring neighborhoods

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
toronto_venues = getNearbyVenues(names=df_borough['Neighborhood'],
                                   latitudes=df_borough['Latitude'],
                                   longitudes=df_borough['Longitude']
                                  )

Regent Park ,  Harbourfront
Queen's Park ,  Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond ,  Adelaide ,  King
Dufferin ,  Dovercourt Village
Harbourfront East ,  Union Station ,  Toronto Islands
Little Portugal ,  Trinity
The Danforth West ,  Riverdale
Toronto Dominion Centre ,  Design Exchange
Brockton ,  Parkdale Village ,  Exhibition Place
India Bazaar ,  The Beaches West
Commerce Court ,  Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park ,  The Junction South
North Toronto West
The Annex ,  North Midtown ,  Yorkville
Parkdale ,  Roncesvalles
Davisville
University of Toronto ,  Harbord
Runnymede ,  Swansea
Moore Park ,  Summerhill East
Kensington Market ,  Chinatown ,  Grange Park
Summerhill West ,  Rathnelly ,  South Hill ,  Forest Hill SE ,  Deer Park
CN Tower ,  King and Spadina ,  Railway Lands ,  Harbourfront West ,  Bathurst Quay ,  South

In [32]:
print(toronto_venues.shape)
toronto_venues.head()

(1604, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park , Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [33]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
"Brockton , Parkdale Village , Exhibition Place",24,24,24,24,24,24
Business reply mail Processing CentrE,16,16,16,16,16,16
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",16,16,16,16,16,16
Central Bay Street,60,60,60,60,60,60
Christie,16,16,16,16,16,16
Church and Wellesley,77,77,77,77,77,77
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


#### Let's find out how many unique categories can be curated from all the returned venues

In [34]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 228 uniques categories.


## Analyze Each Neighborhood

In [35]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
toronto_onehot.shape

(1604, 228)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [37]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000
1,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business reply mail Processing CentrE,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower , King and Spadina , Railway Lands ...",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.016667,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.016667,0.000000,0.000000,0.000000,0.000000
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.025974,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012987,0.000000,...,0.012987,0.012987,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court , Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.030303,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


In [38]:
toronto_grouped.shape

(39, 228)

#### Let's print each neighborhood along with the top 5 most common venues

In [39]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2                Café  0.04
3  Seafood Restaurant  0.04
4         Cheese Shop  0.04


----Brockton ,  Parkdale Village ,  Exhibition Place----
               venue  freq
0               Café  0.12
1          Nightclub  0.12
2     Breakfast Spot  0.08
3        Coffee Shop  0.08
4  Convenience Store  0.04


----Business reply mail Processing CentrE----
                  venue  freq
0  Gym / Fitness Center  0.06
1         Garden Center  0.06
2            Comic Shop  0.06
3           Pizza Place  0.06
4      Recording Studio  0.06


----CN Tower ,  King and Spadina ,  Railway Lands ,  Harbourfront West ,  Bathurst Quay ,  South Niagara ,  Island airport----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4           Airport  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop

#### Let's put that into a *pandas* dataframe

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Seafood Restaurant,Restaurant,Cheese Shop,Café,Jazz Club,Shopping Mall
1,"Brockton , Parkdale Village , Exhibition Place",Nightclub,Café,Breakfast Spot,Coffee Shop,Climbing Gym,Bar,Stadium,Bakery,Intersection,Italian Restaurant
2,Business reply mail Processing CentrE,Park,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Burrito Place,Restaurant,Brewery,Light Rail Station,Smoke Shop
3,"CN Tower , King and Spadina , Railway Lands ...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Bar,Coffee Shop,Sculpture Garden,Airport
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Bar,Ice Cream Shop,Salad Place,Thai Restaurant,Bubble Tea Shop


## 4. Cluster Neighborhoods

In [42]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [43]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_borough

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Café,Breakfast Spot,Theater,Ice Cream Shop,Chocolate Shop,Spa
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Creperie,Sandwich Place,Burger Joint,Burrito Place,Café,Park
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Restaurant,Japanese Restaurant,Bubble Tea Shop,Cosmetics Shop,Ramen Restaurant,Bookstore
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Cocktail Bar,Gastropub,American Restaurant,Hotel,Creperie,Lingerie Store,Department Store,Moroccan Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Coffee Shop,Pub,Trail,Women's Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Distribution Center


In [44]:
# create map
latitude = 43.6532
longitude = -79.3832
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters